#### Overview

| Detail Tag | Information |
|------------|-------------|
|Script Name | bio_helper_utils |
|Developed By | Shankar Hadimani |
|Input Datasets |<ul><li>N/A</li></ul>|
|Output Datasets |<ul><li>N/A</li></ul>|
|Input Data Source |N/A |
|Output Data Source |Code Orange 2.0 Delta-lake |

#### History

| Date | Created/Modified by | Description |
|:----:|--------------|--------|
|23-Feb-2022 | Shankar Hadimani |Initial version |


#### Other Details
- This notebook sets helper functions for - retry, logging, watermark table and etc.

### Import helper functions from utils

In [0]:
### Import libraries, required
from multiprocessing.pool import ThreadPool
from functools import wraps
from pyspark.sql.functions import lit, monotonically_increasing_id, row_number, max, count,col
from pyspark.sql.functions import current_date, current_timestamp
from pyspark.sql.window import Window
from datetime import datetime, timedelta
from requests_ntlm2 import HttpNtlmAuth
from pyspark.sql import DataFrame
from pyspark.sql.types import TimestampType, StringType
from delta.tables import DeltaTable
from urllib.parse import quote, unquote, quote_plus
import pyspark.sql.functions as psf
import requests
import json
import urllib3
import unicodedata
import re
import logging
import time



### disable warning from URL
urllib3.disable_warnings()

In [0]:
def read_data_ntlm_url(url:str, 
                       username:str, 
                       passkey: str) -> dict:
    """This function returns JSON response from an API endpoint
    using ntlm authentication.
    Args:
        url (str): URL of an API endpoint
        username (str): Username for authentication
        passkey (str): password for authentication
    Returns:
        dict: returns JSON response as a dictionary
    """
    session = requests.Session()
    x_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0',
                   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q = 0.8',
                   'Accept-Encoding': 'gzip, deflate',
                   'Connection': 'keep-alive',
                   'Cache-Control': 'max-age=0',
                   'Upgrade-Insecure-Requests': '1'
                   }
    session.headers = x_headers
    session.auth = HttpNtlmAuth(username,passkey)
    response = session.get(url, verify=False)
    return response.status_code, response.text


def create_dataframe(data:dict)-> DataFrame:
    """This function generates a spark dataframe out of jSN dictionary
    Args:
        data (dict): Input shoudl be of class Dictionary
    Returns:
        DataFrame: Spark DataFrame is returned
    """
    jsonRDD = sc.parallelize([data])
    df = spark.read.json(jsonRDD)
    return df

def normalize(column: str) -> str:
    """Normalize column name by replacing invalid characters with underscore
    strips accents and make lowercase
    Args:
        column (str): column name
    Returns:
        str: normalized column name
    """
    n = re.sub(r"[ ,;{}()\n\t=]+", '_', column.lower())
    return unicodedata.normalize('NFKD', n).encode('ASCII', 'ignore').decode()

    
def file_exists(path:str) -> bool:
    """ Function to check for file existence on datalake
    Args:
        path (str): path from ADLS Gen2
    Returns:
        boolean: True/False
    """
    if path[:5] == "/dbfs":
        return os.path.exists(path)
    else:
        try:
            dbutils.fs.ls(path)
            return True
        except Exception as e:
            if 'java.io.FileNotFoundException' in str(e):
                return False
            else:
                raise
                

def update_watermark(source:str,
                    table_name: str,
                     col_name: str,
                    api_end_date:str,
                    p_watermark_fdn_lcn:str,
                    watermark_dbname:str,
                    watermark_table: str,
                    logger,
                    plant_code:str = 'N/A',
                    job_instance:str= 'N/A',
                    ) -> None:
  
  ### create foundation interface external table for TAG Values table
  key_value_dict = {'source':source,
                    'plant_code': plant_code,
                     'table_name':table_name,
                     'column_name':col_name,
                     'value':api_end_date,
                   'job_instance':job_instance}
  ### create dataframe
  df = spark.read.json(sc.parallelize([key_value_dict]))
  df = (df
        .withColumn("load_start_datetime",psf.current_timestamp())
        .withColumn("load_end_datetime",psf.current_timestamp())
       )
  try:
    if DeltaTable.isDeltaTable(spark, p_watermark_fdn_lcn):
      deltaTable = DeltaTable.forPath(spark, p_watermark_fdn_lcn)
      
      # Schema Evolution with a Merge Operation
      deltaTable.alias("t") \
                .merge(df.alias("s"), 
                                "t.source = s.source \
                                AND t.plant_code=s.plant_code \
                                AND t.table_name=s.table_name \
                                AND t.job_instance=s.job_instance") \
           .whenMatchedUpdate(set={ i:'s.'+i for i in ['value','load_end_datetime'] } ) \
           .whenNotMatchedInsertAll() \
           .execute()
    else:
      ### write raw and final result onto foundation layer
      df.write.format('delta') \
      .mode("overwrite") \
      .option("mergeSchema", "true") \
      .partitionBy("source","plant_code","table_name","job_instance") \
      .option("path", p_watermark_fdn_lcn) \
      .saveAsTable(watermark_dbname +'.'+ watermark_table)
  except Exception as e:
    msg ='INFO: concurrent ACID transactions might have conflicts..: '+ str(e)
    log(msg, logger)
  return None
    
    

def retry(exceptions, total_tries=4, initial_wait=0.5, backoff_factor=2, logger=None):
    """
    calling the decorated function applying an exponential backoff.
    Args:
        exceptions: Exception(s) that trigger a retry, can be a tuple
        total_tries: Total tries
        initial_wait: Time to first retry
        backoff_factor: Backoff multiplier (e.g. value of 2 will double the delay each retry).
        logger: logger to be used, if none specified print
    """
    def retry_decorator(f):
        @wraps(f)
        def func_with_retries(*args, **kwargs):
            _tries, _delay = total_tries + 1, initial_wait
            while _tries > 1:
                try:
                    #log(f'{total_tries + 2 - _tries}. try:', logger)
                    return f(*args, **kwargs)
                except exceptions as e:
                    _tries -= 1
                    print_args = args[:1] if args else 'no args'
                    if _tries == 1:
                        msg = str(f'Function: {f.__name__}\n'
                                  f'Exception: {e}\n'
                                  f'Failed despite best efforts after {total_tries} tries.\n'
                                  f' Initial few args: {print_args}, kwargs: {kwargs}')
                        log(msg, logger)
                        raise
                        return f(*args, **kwargs)
                    msg = str(f'Function: {f.__name__}\n'
                              f'Exception: {e}\n'
                              f'Retrying in {_delay} seconds!, args: {print_args}, kwargs: {kwargs}\n')
                    log(msg, logger)
                    time.sleep(_delay)
                    _delay *= backoff_factor

        return func_with_retries
    return retry_decorator
        
def setlogger(app_name=None, level=logging.INFO):
  ### set logger
  msg_format= '%(asctime)s %(levelname)s %(name)s: %(message)s'
  date_format = '%Y-%m-%dT%H:%M:%S - %Z'
  logging.basicConfig(format=msg_format, datefmt=date_format)
  logger = logging.getLogger(app_name)
  logger.setLevel(level)
  return logger
  

def log(msg, logger=None):
  curr_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
  msg = ' ==>> '+ msg
  if logger:
    logger.warning(msg)
  else:
    print(msg)
    
    
def get_json_response(source_raw_lcn:str, 
                      dest_raw_lcn:str, 
                      json_response:dict):
  try:
    dbutils.fs.put(source_raw_lcn, json_response, True)
    dbutils.fs.cp(source_raw_lcn, dest_raw_lcn)
    dbutils.fs.rm(source_raw_lcn,True)
  except Exception as e:
    raise ValueError(e)
    

def configure_mesh_node(scope_name: str = None,
                        adls_name_secret: str = None,
                        tenant_id_secret: str = None,
                        client_id_secret: str = None,
                        client_key_secret: str = None):
  
  # get secret from key vault
  ent_adls_name = dbutils.secrets.get(scope=scope_name,key=adls_name_secret)
  tenant_id = dbutils.secrets.get(scope=scope_name,key=tenant_id_secret)
  client_id = dbutils.secrets.get(scope=scope_name,key=client_id_secret)
  client_key = dbutils.secrets.get(scope=scope_name,key=client_key_secret)
  
  # set spark configuration
  spark.conf.set("fs.azure.account.auth.type.{adls}.dfs.core.windows.net".format(adls=ent_adls_name), "OAuth")
  spark.conf.set("fs.azure.account.oauth.provider.type.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),
                 "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
  spark.conf.set("fs.azure.account.oauth2.client.endpoint.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),
                 "https://login.microsoftonline.com/{tenant}/oauth2/token".format(tenant=tenant_id))
  spark.conf.set("fs.azure.account.oauth2.client.id.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),client_id)
  spark.conf.set("fs.azure.account.oauth2.client.secret.{adls_name}.dfs.core.windows.net".format(adls_name=ent_adls_name),client_key)
  return None

#URL Encode
def urlencode(value):
  safe_string = value.replace('.','%2E')
  return quote_plus(safe_string, safe="")

udf_urlencode = udf(urlencode, StringType())

#URL Decode
@udf(returnType=StringType())
def urldecode(value):
  return unquote(value).replace('%2E','.')

udf_urldecode= udf(urldecode, StringType())